#### Baseline:
Qwen2.5-Qwen2.5-Coder-7B-Instruct


📊 Accuracy by Task Type:

  Task Type  Accuracy
measurement    35.42%
 arithmetic    19.48%
    numbers    12.24%
 comparison    11.54%
    algebra    10.49%
   calculus     5.26%
polynomials     2.63%
probability     0.00%


/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_93006/3788320724.py:53: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=accuracy_by_task_type.values, y=accuracy_by_task_type.index, palette='viridis')


🔹 Total tasks: 1214
🔹 Raw accuracy: 12.19%
🔹 Adjusted accuracy (with float match errors): 13.67%
🔹 Float match errors (marked incorrect): 18
🔹 Empty answers: 0


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../results/baseline/baseline_results.csv')

df['task_type'] = df['file'].apply(lambda x: x.split('__')[0] if '__' in x else x.split('.')[0])

def try_parse_float(value):
    try:
        return float(value)
    except:
        return None

df['gt_float'] = df['gt_answer'].apply(try_parse_float)
df['llm_float'] = df['llm_answer'].apply(try_parse_float)

total_tasks = len(df)

overall_accuracy = df['correct'].mean()
accuracy_by_task_type = df.groupby('task_type')['correct'].mean().sort_values(ascending=False)

float_match_errors = df[
    (~df['correct']) &
    (df['gt_float'].notna()) &
    (df['llm_float'].notna()) &
    (np.isclose(df['gt_float'], df['llm_float'], atol=1e-4))
]
num_float_match_errors = len(float_match_errors)

adjusted_correct = df['correct'].sum() + num_float_match_errors
adjusted_accuracy = adjusted_correct / total_tasks

num_empty_answers = df['llm_answer'].isna().sum() + (df['llm_answer'].astype(str).str.strip() == '').sum()

accuracy_table = accuracy_by_task_type.reset_index()
accuracy_table.columns = ['Task Type', 'Accuracy']
print("\n📊 Accuracy by Task Type:\n")
print(accuracy_table.to_string(index=False, float_format="{:.2%}".format))

plt.figure(figsize=(12, 6))
sns.barplot(x=accuracy_by_task_type.values, y=accuracy_by_task_type.index, palette='viridis')
plt.xlabel('Accuracy')
plt.ylabel('Task Type')
plt.title('Accuracy by Task Type')
plt.grid(True)
plt.tight_layout()
plt.show()

print(f'\n🔹 Total tasks: {total_tasks}')
print(f'🔹 Raw accuracy: {overall_accuracy:.2%}')
print(f'🔹 Adjusted accuracy (with float match errors): {adjusted_accuracy:.2%}')
print(f'🔹 Float match errors (marked incorrect): {num_float_match_errors}')
print(f'🔹 Empty answers: {num_empty_answers}')


#### Agent with code execution

In [ ]:
import importlib.util
import sys
import os
import glob
import matplotlib.pyplot as plt

script_path = "../eval_agent/get_error_statistic.py"
module_name = "get_error_statistic"

spec = importlib.util.spec_from_file_location(module_name, script_path)
module = importlib.util.module_from_spec(spec)
sys.modules[module_name] = module
spec.loader.exec_module(module)


In [40]:
folder = "../results/code_eval"
prefix = "train_"
pattern = os.path.join(folder, f"{prefix}*.json")
files = glob.glob(pattern)

print(f"🔍 Found: {len(files)}")

from collections import defaultdict

category_totals = defaultdict(lambda: defaultdict(int))

for filepath in files:
    stats, cuda_oom, too_many_steps, digits_error = module.analyze_file(filepath)
    for cat, data in stats.items():
        for key, value in data.items():
            category_totals[cat][key] += value


🔍 Found: 4


/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_93006/2387955351.py:29: UserWarning: Glyph 129518 (\N{ABACUS}) missing from font(s) Arial.
  plt.tight_layout()
/Users/diana/Library/Python/3.13/lib/python/site-packages/IPython/core/pylabtools.py:170: UserWarning: Glyph 129518 (\N{ABACUS}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)


/var/folders/gg/wqlmrvl91mj28lsltv0ccxqm0000gn/T/ipykernel_93006/2387955351.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_table['total_errors'] = numeric_table[error_types].sum(axis=1)


,errors,cuda_oom,digits_error,total,total_errors
numbers,67,16,9,173,92
arithmetic,34,3,36,141,73
algebra,54,5,6,82,65
comparison,58,4,0,102,62
polynomials,35,7,1,72,43
probability,37,0,1,38,38
measurement,6,0,11,28,17
calculus,6,1,0,18,7


In [41]:
from IPython.display import display


df_stats = pd.DataFrame.from_dict(category_totals, orient='index')
df_stats = df_stats.sort_values('total', ascending=False)


error_types = ['errors', 'cuda_oom', 'digits_error']
df_stats_plot = df_stats[error_types].copy()
df_stats_plot['category'] = df_stats_plot.index
df_stats_plot = df_stats_plot.reset_index(drop=True)


plt.figure(figsize=(14, 6))
barplot = sns.barplot(
    data=pd.melt(df_stats_plot, id_vars='category', value_vars=error_types),
    x='category', y='value', hue='variable', palette='tab10'
)
plt.title("🧮 Errors by categories", fontsize=16)
plt.xlabel("Category")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.6)


for container in barplot.containers:
    barplot.bar_label(container, fmt='%d', label_type='edge', fontsize=9)

plt.tight_layout()
plt.legend(title='Error type')
plt.show()


numeric_table = df_stats[error_types + ['total']]
numeric_table['total_errors'] = numeric_table[error_types].sum(axis=1)
numeric_table = numeric_table.sort_values('total_errors', ascending=False)

display(numeric_table)


In [42]:
total_tasks = df_stats['total'].sum()
total_correct = df_stats['total'].sum() - df_stats['errors'].sum()
total_correct_with_equiv = total_correct + df_stats['digits_error'].sum()


accuracy = total_correct / total_tasks * 100
accuracy_with_equiv = total_correct_with_equiv / total_tasks * 100

print(f"✅ Total number: {total_tasks}")
print(f"✅ Correct answers: {total_correct} ({accuracy:.2f}%)")
print(f"✅ Correct with equivivalence for 25.0 and 25: {total_correct_with_equiv} ({accuracy_with_equiv:.2f}%)")


✅ Total number: 654
✅ Correct answers: 357 (54.59%)
✅ Correct with equivivalence for 25.0 and 25: 421 (64.37%)


In [31]:
import os, json
from math import isclose
from collections import defaultdict

def is_equivalent(a, b, tol=1e-6):
    try:
        return isclose(float(a), float(b), abs_tol=tol)
    except:
        return str(a).strip() == str(b).strip()


eval_folder = "../results/code_eval"
eval_files = glob.glob(os.path.join(eval_folder, "math_*.json"))

print(f"Found files: {len(eval_files)}")

# Статистика по всем файлам
overall_stats = defaultdict(int)

for file in eval_files:
    with open(file, "r") as f:
        data = json.load(f)
    
    stats = data.get("statistics", {})
    details = stats.get("details", [])

    overall_stats["total"] += stats.get("total", len(details))
    overall_stats["correct"] += stats.get("correct", 0)
    overall_stats["incorrect"] += stats.get("incorrect", 0)
    overall_stats["no_answer"] += stats.get("no_answer", 0)
    overall_stats["api_errors"] += stats.get("api_errors", 0)

    # Добавим "essentially correct"
    for item in details:
        expected = item.get("expected", "").strip()
        predicted = item.get("predicted", "").strip()
        verdict = item.get("verdict", "").upper()

        if verdict == "CORRECT":
            overall_stats["essentially_correct"] += 1
        elif is_equivalent(expected, predicted):
            overall_stats["essentially_correct"] += 1


Found files: 4


In [34]:
print("Statistic:")
for k, v in overall_stats.items():
    print(f"{k:25} {v}")


total = overall_stats["total"]
percent_correct = overall_stats["correct"] / total * 100

print(f"\n✅ Correct: {percent_correct:.2f}%")


Statistic:
total                     654
correct                   464
incorrect                 190
no_answer                 0
api_errors                0
essentially_correct       466

✅ Correct: 70.95%


#### Agent with code execution + feedback